In [ ]:
import pandas as pd
import folium
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import os

def find_project_root():
    """Find project root by looking for the data folder."""
    current_dir = os.getcwd()
    while current_dir != os.path.dirname(current_dir):  # Until filesystem root
        data_path = os.path.join(current_dir, 'data')
        if os.path.exists(data_path):
            return current_dir
        current_dir = os.path.dirname(current_dir)
    return os.getcwd()

class BudapestRouteVisualizer:
    def __init__(self, data_folder=None):
        """
        Initialize the visualizer with the Budapest time travel data structure
        
        If data_folder is None, automatically detect the project root and use data/processed/
        """
        if data_folder is None:
            project_root = find_project_root()
            data_folder = os.path.join(project_root, 'data', 'processed')
        
        self.data_folder = data_folder
        
        # Load the CSV files
        self.routes = pd.read_csv(os.path.join(data_folder, 'routes.csv'))
        self.route_versions = pd.read_csv(os.path.join(data_folder, 'route_versions.csv'))
        self.shapes = pd.read_csv(os.path.join(data_folder, 'shapes.csv'))
        
        # Optional files that might not exist
        try:
            self.shape_variants = pd.read_csv(os.path.join(data_folder, 'shape_variants.csv'))
        except FileNotFoundError:
            self.shape_variants = pd.DataFrame()
            
        try:
            self.shape_variant_activations = pd.read_csv(os.path.join(data_folder, 'shape_variant_activations.csv'))
        except FileNotFoundError:
            self.shape_variant_activations = pd.DataFrame()
        
        # Convert date columns - try different date formats
        date_columns = ['start_date', 'end_date', 'activation_date', 'deactivation_date', 
                       'from_date', 'to_date', 'valid_from', 'valid_to', 'date_start', 'date_end']
        for df in [self.route_versions, self.shape_variant_activations]:
            if not df.empty:
                for col in date_columns:
                    if col in df.columns:
                        df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Setup widgets
        self.setup_widgets()
        
    def setup_widgets(self):
        """Setup interactive widgets"""
        # Initialize map bounds for zoom preservation
        self.current_bounds = None
        self.current_zoom = 12
        
        # Route selector - handle different possible column names
        route_short_col = 'route_short_name' if 'route_short_name' in self.routes.columns else 'short_name'
        route_long_col = 'route_long_name' if 'route_long_name' in self.routes.columns else 'long_name'
        route_id_col = 'route_id' if 'route_id' in self.routes.columns else 'id'
        
        route_options = []
        for _, row in self.routes.iterrows():
            route_id = row[route_id_col]
            short_name = row.get(route_short_col, str(route_id))
            long_name = row.get(route_long_col, '')
            
            display_name = f"{short_name}" + (f" - {long_name}" if long_name else "")
            route_options.append((display_name, route_id))
        
        self.route_selector = widgets.Dropdown(
            options=route_options,
            value=route_options[0][1] if route_options else None,
            description='Route:',
            style={'description_width': 'initial'}
        )
        
        # Date slider (will be updated when route is selected)
        self.date_slider = widgets.SelectionSlider(
            options=[datetime.now().date()],
            value=datetime.now().date(),
            description='Date:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            style={'description_width': 'initial'}
        )
        
        # Set up event handlers
        self.route_selector.observe(self.on_route_change, names='value')
        self.date_slider.observe(self.on_date_change, names='value')
        
        # Initialize with first route
        if route_options:
            self.on_route_change({'new': route_options[0][1]})
    
    def get_route_lifetime(self, route_id):
        """Get the lifetime dates for a specific route"""
        route_data = self.route_versions[self.route_versions['route_id'] == route_id]
        if route_data.empty:
            return None, None
        
        # Handle different possible date column names
        start_col = None
        end_col = None
        
        possible_start_cols = ['start_date', 'from_date', 'valid_from', 'date_start']
        possible_end_cols = ['end_date', 'to_date', 'valid_to', 'date_end']
        
        for col in possible_start_cols:
            if col in route_data.columns:
                start_col = col
                break
                
        for col in possible_end_cols:
            if col in route_data.columns:
                end_col = col
                break
        
        if not start_col or not end_col:
            return None, None
        
        start_date = route_data[start_col].min()
        end_date = route_data[end_col].max()
        return start_date, end_date
    
    def get_available_dates(self, route_id):
        """Get all dates when the route had different configurations"""
        route_data = self.route_versions[self.route_versions['route_id'] == route_id]
        
        # Handle different possible date column names
        start_col = None
        end_col = None
        
        possible_start_cols = ['start_date', 'from_date', 'valid_from', 'date_start']
        possible_end_cols = ['end_date', 'to_date', 'valid_to', 'date_end']
        
        for col in possible_start_cols:
            if col in route_data.columns:
                start_col = col
                break
                
        for col in possible_end_cols:
            if col in route_data.columns:
                end_col = col
                break
        
        if not start_col or not end_col:
            return [datetime.now().date()]
        
        dates = set()
        for _, row in route_data.iterrows():
            current_date = row[start_col]
            end_date = row[end_col]
            
            if pd.isna(current_date) or pd.isna(end_date):
                continue
                
            # Add key dates: start, end, and some intermediate dates
            dates.add(current_date.date())
            dates.add(end_date.date())
            
            # Add monthly intervals for long-running routes
            while current_date < end_date:
                dates.add(current_date.date())
                current_date += timedelta(days=30)
        
        return sorted(list(dates)) if dates else [datetime.now().date()]
    
    def get_main_shape_for_date(self, route_id, target_date):
        """Get the main_shape_id for a specific route on a specific date"""
        route_data = self.route_versions[self.route_versions['route_id'] == route_id]
        
        # Handle different possible date column names
        start_col = None
        end_col = None
        
        possible_start_cols = ['start_date', 'from_date', 'valid_from', 'date_start']
        possible_end_cols = ['end_date', 'to_date', 'valid_to', 'date_end']
        
        for col in possible_start_cols:
            if col in route_data.columns:
                start_col = col
                break
                
        for col in possible_end_cols:
            if col in route_data.columns:
                end_col = col
                break
        
        if not start_col or not end_col:
            return None
        
        # Convert target_date to datetime if it's a date
        if hasattr(target_date, 'date'):
            target_date = target_date
        else:
            target_date = datetime.combine(target_date, datetime.min.time())
        
        # Find the version active on the target date
        active_version = route_data[
            (route_data[start_col] <= target_date) & 
            (route_data[end_col] >= target_date)
        ]
        
        if not active_version.empty:
            # Handle different possible shape column names
            shape_col = 'main_shape_id'
            if 'main_shape_id' not in active_version.columns:
                possible_shape_cols = ['shape_id', 'main_shape', 'shape']
                for col in possible_shape_cols:
                    if col in active_version.columns:
                        shape_col = col
                        break
            
            if shape_col in active_version.columns:
                return active_version.iloc[0][shape_col]
        return None
    
    def get_route_color(self, route_id):
        """Get the color for a specific route from routes.csv"""
        route_info = self.routes[self.routes['route_id'] == route_id]
        if route_info.empty:
            return '#0000FF'  # Default blue
        
        route_info = route_info.iloc[0]
        
        # Try different possible color column names
        color_cols = ['route_color', 'color', 'route_colour', 'colour']
        for col in color_cols:
            if col in route_info and pd.notna(route_info[col]):
                color = str(route_info[col])
                # Add # if not present
                if not color.startswith('#'):
                    color = '#' + color
                return color
        
        return '#0000FF'  # Default blue if no color found
    def get_shape_coordinates(self, shape_id):
        """Get coordinates for a specific shape"""
        if shape_id is None:
            return []
        
        shape_data = self.shapes[self.shapes['shape_id'] == shape_id].sort_values('shape_pt_sequence')
        coordinates = [(row['shape_pt_lat'], row['shape_pt_lon']) for _, row in shape_data.iterrows()]
        return coordinates
    
    def create_map(self, route_id, target_date):
        """Create a folium map for the route on the specified date"""
        # Get main shape for the date
        main_shape_id = self.get_main_shape_for_date(route_id, target_date)
        coordinates = self.get_shape_coordinates(main_shape_id)
        
        if not coordinates:
            # Create empty map centered on Budapest
            m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)
            folium.Marker(
                [47.4979, 19.0402],
                popup="No route data available for this date",
                icon=folium.Icon(color='red')
            ).add_to(m)
            return m
        
        # Calculate center point
        center_lat = sum(coord[0] for coord in coordinates) / len(coordinates)
        center_lon = sum(coord[1] for coord in coordinates) / len(coordinates)
        
        # Create map with preserved zoom level
        if self.current_bounds:
            m = folium.Map(location=[center_lat, center_lon])
            m.fit_bounds(self.current_bounds)
        else:
            m = folium.Map(location=[center_lat, center_lon], zoom_start=self.current_zoom)
        
        # Get route info and color
        route_info = self.routes[self.routes['route_id'] == route_id]
        route_color = self.get_route_color(route_id)
        
        if route_info.empty:
            route_name = f"Route {route_id}"
        else:
            route_info = route_info.iloc[0]
            # Handle different possible column names
            short_name = route_info.get('route_short_name', route_info.get('short_name', str(route_id)))
            long_name = route_info.get('route_long_name', route_info.get('long_name', ''))
            route_name = f"{short_name}" + (f" - {long_name}" if long_name else "")
        
        # Add route line with the route's actual color
        folium.PolyLine(
            coordinates,
            color=route_color,
            weight=4,
            opacity=0.8,
            popup=f"{route_name} on {target_date}"
        ).add_to(m)
        
        # Add start and end markers
        if len(coordinates) >= 2:
            folium.Marker(
                coordinates[0],
                popup=f"Start: {route_name}",
                icon=folium.Icon(color='green', icon='play')
            ).add_to(m)
            
            folium.Marker(
                coordinates[-1],
                popup=f"End: {route_name}",
                icon=folium.Icon(color='red', icon='stop')
            ).add_to(m)
        
        return m
    
    def on_route_change(self, change):
        """Handle route selection change"""
        route_id = change['new']
        
        # Update date slider options
        available_dates = self.get_available_dates(route_id)
        if available_dates:
            self.date_slider.options = available_dates
            self.date_slider.value = available_dates[0]
        
        # Update visualization
        self.update_visualization()
    
    def on_date_change(self, change):
        """Handle date selection change"""
        # Store current map bounds before updating
        self.store_current_bounds()
        self.update_visualization()
    
    def store_current_bounds(self):
        """Store current map bounds (would need to be implemented with JavaScript callback in real notebook)"""
        # In a real implementation, you would use JavaScript callbacks to get current map bounds
        # For now, we'll preserve the zoom level
        pass
    
    def update_visualization(self):
        """Update the map visualization"""
        route_id = self.route_selector.value
        target_date = self.date_slider.value
        
        if route_id is None:
            return
        
        # Create and display map
        map_obj = self.create_map(route_id, target_date)
        
        # Display route lifetime info
        start_date, end_date = self.get_route_lifetime(route_id)
        route_info = self.routes[self.routes['route_id'] == route_id]
        
        if not route_info.empty:
            route_info = route_info.iloc[0]
            short_name = route_info.get('route_short_name', route_info.get('short_name', str(route_id)))
            long_name = route_info.get('route_long_name', route_info.get('long_name', ''))
            route_name = f"{short_name}" + (f" - {long_name}" if long_name else "")
        else:
            route_name = f"Route {route_id}"
        
        if start_date and end_date:
            pass  # Keep lifetime info silent
        
        # Show main shape info
        main_shape_id = self.get_main_shape_for_date(route_id, target_date)
        
        return map_obj
    
    def display(self):
        """Display the interactive visualization"""
        display(widgets.VBox([
            self.route_selector,
            self.date_slider
        ]))
        
        # Create output widget for the map
        out = widgets.Output()
        display(out)
        
        def update_display(*args):
            with out:
                clear_output(wait=True)
                map_obj = self.update_visualization()
                if map_obj:
                    display(map_obj)
        
        # Connect the update function
        self.route_selector.observe(update_display, names='value')
        self.date_slider.observe(update_display, names='value')
        
        # Initial display
        update_display()

# Usage example with automatic folder detection:

# Initialize the visualizer (automatically finds data folder)
visualizer = BudapestRouteVisualizer()

# Display the interactive visualization
visualizer.display()


C:\Users\User\AppData\Local\Temp\ipykernel_4264\3271062621.py:35: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  self.shapes = pd.read_csv(os.path.join(data_folder, 'shapes.csv'))


Output()

In [29]:
import pandas as pd
import folium
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import os

def find_project_root():
    """Find project root by looking for the data folder."""
    current_dir = os.getcwd()
    while current_dir != os.path.dirname(current_dir):  # Until filesystem root
        data_path = os.path.join(current_dir, 'data')
        if os.path.exists(data_path):
            return current_dir
        current_dir = os.path.dirname(current_dir)
    return os.getcwd()

class BudapestRouteVisualizer:
    def __init__(self, data_folder=None):
        """
        Initialize the visualizer with the Budapest time travel data structure
        
        If data_folder is None, automatically detect the project root and use data/processed/
        """
        if data_folder is None:
            project_root = find_project_root()
            data_folder = os.path.join(project_root, 'data', 'processed')
        
        self.data_folder = data_folder
        
        # Load the CSV files
        self.routes = pd.read_csv(os.path.join(data_folder, 'routes.csv'))
        self.route_versions = pd.read_csv(os.path.join(data_folder, 'route_versions.csv'))
        self.shapes = pd.read_csv(os.path.join(data_folder, 'shapes.csv'))
        
        # Optional files that might not exist
        try:
            self.shape_variants = pd.read_csv(os.path.join(data_folder, 'shape_variants.csv'))
        except FileNotFoundError:
            self.shape_variants = pd.DataFrame()
            
        try:
            self.shape_variant_activations = pd.read_csv(os.path.join(data_folder, 'shape_variant_activations.csv'))
        except FileNotFoundError:
            self.shape_variant_activations = pd.DataFrame()
        
        # Convert date columns - try different date formats
        date_columns = ['start_date', 'end_date', 'activation_date', 'deactivation_date', 
                       'from_date', 'to_date', 'valid_from', 'valid_to', 'date_start', 'date_end']
        for df in [self.route_versions, self.shape_variant_activations]:
            if not df.empty:
                for col in date_columns:
                    if col in df.columns:
                        df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Setup widgets
        self.setup_widgets()
        
    def setup_widgets(self):
        """Setup interactive widgets"""
        # Initialize map bounds for zoom preservation
        self.current_bounds = None
        self.current_zoom = 12
        
        # Route selector - handle different possible column names
        route_short_col = 'route_short_name' if 'route_short_name' in self.routes.columns else 'short_name'
        route_long_col = 'route_long_name' if 'route_long_name' in self.routes.columns else 'long_name'
        route_id_col = 'route_id' if 'route_id' in self.routes.columns else 'id'
        
        route_options = []
        for _, row in self.routes.iterrows():
            route_id = row[route_id_col]
            short_name = row.get(route_short_col, str(route_id))
            long_name = row.get(route_long_col, '')
            
            display_name = f"{short_name}" + (f" - {long_name}" if long_name else "")
            route_options.append((display_name, route_id))
        
        self.route_selector = widgets.Dropdown(
            options=route_options,
            value=route_options[0][1] if route_options else None,
            description='Route:',
            style={'description_width': 'initial'}
        )
        
        # Date slider (will be updated when route is selected)
        self.date_slider = widgets.SelectionSlider(
            options=[datetime.now().date()],
            value=datetime.now().date(),
            description='Date:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            style={'description_width': 'initial'}
        )
        
        # Set up event handlers
        self.route_selector.observe(self.on_route_change, names='value')
        self.date_slider.observe(self.on_date_change, names='value')
        
        # Initialize with first route
        if route_options:
            self.on_route_change({'new': route_options[0][1]})
    
    def get_route_lifetime(self, route_id):
        """Get the lifetime dates for a specific route"""
        route_data = self.route_versions[self.route_versions['route_id'] == route_id]
        if route_data.empty:
            return None, None
        
        # Handle different possible date column names
        start_col = None
        end_col = None
        
        possible_start_cols = ['start_date', 'from_date', 'valid_from', 'date_start']
        possible_end_cols = ['end_date', 'to_date', 'valid_to', 'date_end']
        
        for col in possible_start_cols:
            if col in route_data.columns:
                start_col = col
                break
                
        for col in possible_end_cols:
            if col in route_data.columns:
                end_col = col
                break
        
        if not start_col or not end_col:
            return None, None
        
        start_date = route_data[start_col].min()
        end_date = route_data[end_col].max()
        return start_date, end_date
    
    def get_available_dates(self, route_id):
        """Get all dates when the route had different configurations"""
        route_data = self.route_versions[self.route_versions['route_id'] == route_id]
        
        # Handle different possible date column names
        start_col = None
        end_col = None
        
        possible_start_cols = ['start_date', 'from_date', 'valid_from', 'date_start']
        possible_end_cols = ['end_date', 'to_date', 'valid_to', 'date_end']
        
        for col in possible_start_cols:
            if col in route_data.columns:
                start_col = col
                break
                
        for col in possible_end_cols:
            if col in route_data.columns:
                end_col = col
                break
        
        if not start_col or not end_col:
            return [datetime.now().date()]
        
        dates = set()
        for _, row in route_data.iterrows():
            current_date = row[start_col]
            end_date = row[end_col]
            
            if pd.isna(current_date) or pd.isna(end_date):
                continue
                
            # Add key dates: start, end, and some intermediate dates
            dates.add(current_date.date())
            dates.add(end_date.date())
            
            # Add monthly intervals for long-running routes
            while current_date < end_date:
                dates.add(current_date.date())
                current_date += timedelta(days=30)
        
        return sorted(list(dates)) if dates else [datetime.now().date()]
    
    def get_main_shape_for_date(self, route_id, target_date):
        """Get the main_shape_id for a specific route on a specific date"""
        route_data = self.route_versions[self.route_versions['route_id'] == route_id]
        
        # Handle different possible date column names
        start_col = None
        end_col = None
        
        possible_start_cols = ['start_date', 'from_date', 'valid_from', 'date_start']
        possible_end_cols = ['end_date', 'to_date', 'valid_to', 'date_end']
        
        for col in possible_start_cols:
            if col in route_data.columns:
                start_col = col
                break
                
        for col in possible_end_cols:
            if col in route_data.columns:
                end_col = col
                break
        
        if not start_col or not end_col:
            return None
        
        # Convert target_date to datetime if it's a date
        if hasattr(target_date, 'date'):
            target_date = target_date
        else:
            target_date = datetime.combine(target_date, datetime.min.time())
        
        # Find the version active on the target date
        active_version = route_data[
            (route_data[start_col] <= target_date) & 
            (route_data[end_col] >= target_date)
        ]
        
        if not active_version.empty:
            # Handle different possible shape column names
            shape_col = 'main_shape_id'
            if 'main_shape_id' not in active_version.columns:
                possible_shape_cols = ['shape_id', 'main_shape', 'shape']
                for col in possible_shape_cols:
                    if col in active_version.columns:
                        shape_col = col
                        break
            
            if shape_col in active_version.columns:
                return active_version.iloc[0][shape_col]
        return None

    def get_route_change_history(self, route_id):
        """Get the change history for a specific route"""
        route_data = self.route_versions[self.route_versions['route_id'] == route_id].copy()
        
        if route_data.empty:
            return []
        
        # Handle different possible date column names
        start_col = None
        end_col = None
        
        possible_start_cols = ['start_date', 'from_date', 'valid_from', 'date_start']
        possible_end_cols = ['end_date', 'to_date', 'valid_to', 'date_end']
        
        for col in possible_start_cols:
            if col in route_data.columns:
                start_col = col
                break
                
        for col in possible_end_cols:
            if col in route_data.columns:
                end_col = col
                break
        
        if not start_col or not end_col:
            return []
        
        # Sort by start date
        route_data = route_data.sort_values(start_col)
        
        changes = []
        for _, row in route_data.iterrows():
            change = {
                'start_date': row[start_col],
                'end_date': row[end_col],
                'main_shape_id': row.get('main_shape_id', row.get('shape_id', 'Unknown')),
                'version_id': row.get('version_id', row.get('id', 'Unknown')),
                'direction_id': row.get('direction_id', 'Unknown')
            }
            changes.append(change)
        
        return changes

    def get_active_shape_variants(self, route_id, target_date):
        """Get all active shape variants for a route on a specific date"""
        if self.shape_variants.empty or self.shape_variant_activations.empty:
            return []
        
        # Convert target_date to datetime if it's a date
        if hasattr(target_date, 'date'):
            target_date = target_date
        else:
            target_date = datetime.combine(target_date, datetime.min.time())
        
        # Check what columns actually exist
        if 'shape_id' not in self.shape_variants.columns or 'shape_id' not in self.shape_variant_activations.columns:
            # If shape_id columns don't exist, return empty list
            return []
        
        # Check if shape_variants has route_id column
        if 'route_id' not in self.shape_variants.columns:
            # If no route_id in shape_variants, we need to find shapes through route_versions
            # Get the main_shape_id for this route on this date
            main_shape_id = self.get_main_shape_for_date(route_id, target_date)
            if not main_shape_id:
                return []
            
            # Get all shape variants that might be related to this main shape
            # This is a fallback approach - you might need to adjust based on your data structure
            route_variants = self.shape_variants.copy()
        else:
            # Get shape variants for this route directly
            route_variants = self.shape_variants[self.shape_variants['route_id'] == route_id]
        
        if route_variants.empty:
            return []
        
        active_shapes = []
        
        for _, variant in route_variants.iterrows():
            shape_id = variant['shape_id']
            
            # Check if this shape is active on the target date
            activations = self.shape_variant_activations[
                self.shape_variant_activations['shape_id'] == shape_id
            ]
            
            if activations.empty:
                continue
            
            for _, activation in activations.iterrows():
                # Handle different possible date column names
                start_col = None
                end_col = None
                
                possible_start_cols = ['activation_date', 'start_date', 'from_date', 'valid_from']
                possible_end_cols = ['deactivation_date', 'end_date', 'to_date', 'valid_to']
                
                for col in possible_start_cols:
                    if col in activation.index and pd.notna(activation[col]):
                        start_col = col
                        break
                        
                for col in possible_end_cols:
                    if col in activation.index and pd.notna(activation[col]):
                        end_col = col
                        break
                
                # Check if active on target date
                is_active = True
                
                if start_col and pd.notna(activation[start_col]):
                    if target_date < activation[start_col]:
                        is_active = False
                        
                if end_col and pd.notna(activation[end_col]):
                    if target_date > activation[end_col]:
                        is_active = False
                
                # Check exception_type (NaN or 1 means active)
                exception_type = activation.get('exception_type', float('nan'))
                if pd.notna(exception_type) and exception_type != 1:
                    is_active = False
                
                if is_active:
                    shape_info = {
                        'shape_id': shape_id,
                        'exception_type': exception_type,
                        'activation_start': activation.get(start_col, 'Unknown'),
                        'activation_end': activation.get(end_col, 'Unknown')
                    }
                    
                    # Add any additional info from shape_variants
                    for col in variant.index:
                        if col not in ['route_id', 'shape_id']:
                            shape_info[col] = variant[col]
                    
                    active_shapes.append(shape_info)
                    break  # Found active activation for this shape
        
        return active_shapes
    def display_route_details(self, route_id, target_date=None):
        """Display detailed information about the selected route"""
        # Get route basic info
        route_info = self.routes[self.routes['route_id'] == route_id]
        if route_info.empty:
            return f"Route {route_id} - No details available"
        
        route_info = route_info.iloc[0]
        short_name = route_info.get('route_short_name', route_info.get('short_name', str(route_id)))
        long_name = route_info.get('route_long_name', route_info.get('long_name', ''))
        route_type = route_info.get('route_type', 'Unknown')
        route_color = self.get_route_color(route_id)
        
        # Get change history
        changes = self.get_route_change_history(route_id)
        
        # Build details string
        details = []
        details.append(f"🚊 **Route {short_name}**" + (f" - {long_name}" if long_name else ""))
        details.append(f"📍 Type: {route_type}")
        details.append(f"🎨 Color: {route_color}")
        
        if changes:
            details.append(f"\n📅 **Change History ({len(changes)} versions):**")
            for i, change in enumerate(changes, 1):
                start = change['start_date'].strftime('%Y-%m-%d') if pd.notna(change['start_date']) else 'Unknown'
                end = change['end_date'].strftime('%Y-%m-%d') if pd.notna(change['end_date']) else 'Present'
                shape_id = change['main_shape_id']
                version = change['version_id']
                direction = change['direction_id']
                
                details.append(f"  {i}. {start} → {end}")
                details.append(f"     Shape: {shape_id}, Version: {version}, Direction: {direction}")
        else:
            details.append("\n📅 No change history available")
        
        # Get active shape variants for the selected date
        if target_date:
            active_variants = self.get_active_shape_variants(route_id, target_date)
            
            if active_variants:
                details.append(f"\n🔀 **Active Shape Variants on {target_date} ({len(active_variants)} variants):**")
                for i, variant in enumerate(active_variants, 1):
                    shape_id = variant['shape_id']
                    exception_type = variant['exception_type']
                    exception_str = 'NaN' if pd.isna(exception_type) else str(int(exception_type))
                    
                    details.append(f"  {i}. Shape ID: {shape_id}")
                    details.append(f"     Exception Type: {exception_str}")
                    
                    # Add activation period if available
                    start_act = variant.get('activation_start', 'Unknown')
                    end_act = variant.get('activation_end', 'Unknown')
                    if start_act != 'Unknown' or end_act != 'Unknown':
                        start_str = start_act.strftime('%Y-%m-%d') if pd.notna(start_act) else 'Unknown'
                        end_str = end_act.strftime('%Y-%m-%d') if pd.notna(end_act) else 'Present'
                        details.append(f"     Active: {start_str} → {end_str}")
            else:
                details.append(f"\n🔀 **Active Shape Variants on {target_date}:** None found")
        
        return "\n".join(details)

    def get_route_color(self, route_id):
        """Get the color for a specific route from routes.csv"""
        route_info = self.routes[self.routes['route_id'] == route_id]
        if route_info.empty:
            return '#0000FF'  # Default blue
        
        route_info = route_info.iloc[0]
        
        # Try different possible color column names
        color_cols = ['route_color', 'color', 'route_colour', 'colour']
        for col in color_cols:
            if col in route_info and pd.notna(route_info[col]):
                color = str(route_info[col])
                # Add # if not present
                if not color.startswith('#'):
                    color = '#' + color
                return color
        
        return '#0000FF'  # Default blue if no color found

    def get_shape_coordinates(self, shape_id):
        """Get coordinates for a specific shape"""
        if shape_id is None:
            return []
        
        shape_data = self.shapes[self.shapes['shape_id'] == shape_id].sort_values('shape_pt_sequence')
        coordinates = [(row['shape_pt_lat'], row['shape_pt_lon']) for _, row in shape_data.iterrows()]
        return coordinates
    
    def create_map(self, route_id, target_date):
        """Create a folium map for the route on the specified date"""
        # Get main shape for the date
        main_shape_id = self.get_main_shape_for_date(route_id, target_date)
        coordinates = self.get_shape_coordinates(main_shape_id)
        
        if not coordinates:
            # Create empty map centered on Budapest
            m = folium.Map(location=[47.4979, 19.0402], zoom_start=12)
            folium.Marker(
                [47.4979, 19.0402],
                popup="No route data available for this date",
                icon=folium.Icon(color='red')
            ).add_to(m)
            return m
        
        # Calculate center point
        center_lat = sum(coord[0] for coord in coordinates) / len(coordinates)
        center_lon = sum(coord[1] for coord in coordinates) / len(coordinates)
        
        # Create map with preserved zoom level
        if self.current_bounds:
            m = folium.Map(location=[center_lat, center_lon])
            m.fit_bounds(self.current_bounds)
        else:
            m = folium.Map(location=[center_lat, center_lon], zoom_start=self.current_zoom)
        
        # Get route info and color
        route_info = self.routes[self.routes['route_id'] == route_id]
        route_color = self.get_route_color(route_id)
        
        if route_info.empty:
            route_name = f"Route {route_id}"
        else:
            route_info = route_info.iloc[0]
            # Handle different possible column names
            short_name = route_info.get('route_short_name', route_info.get('short_name', str(route_id)))
            long_name = route_info.get('route_long_name', route_info.get('long_name', ''))
            route_name = f"{short_name}" + (f" - {long_name}" if long_name else "")
        
        # Add route line with the route's actual color
        folium.PolyLine(
            coordinates,
            color=route_color,
            weight=4,
            opacity=0.8,
            popup=f"{route_name} on {target_date}"
        ).add_to(m)
        
        # Add start and end markers
        if len(coordinates) >= 2:
            folium.Marker(
                coordinates[0],
                popup=f"Start: {route_name}",
                icon=folium.Icon(color='green', icon='play')
            ).add_to(m)
            
            folium.Marker(
                coordinates[-1],
                popup=f"End: {route_name}",
                icon=folium.Icon(color='red', icon='stop')
            ).add_to(m)
        
        return m
    
    def on_route_change(self, change):
        """Handle route selection change"""
        route_id = change['new']
        
        # Update date slider options
        available_dates = self.get_available_dates(route_id)
        if available_dates:
            self.date_slider.options = available_dates
            self.date_slider.value = available_dates[0]
        
        # Update visualization
        self.update_visualization()
    
    def on_date_change(self, change):
        """Handle date selection change"""
        # Store current map bounds before updating
        self.store_current_bounds()
        self.update_visualization()
    
    def store_current_bounds(self):
        """Store current map bounds (would need to be implemented with JavaScript callback in real notebook)"""
        # In a real implementation, you would use JavaScript callbacks to get current map bounds
        # For now, we'll preserve the zoom level
        pass
    
    def update_visualization(self):
        """Update the map visualization"""
        route_id = self.route_selector.value
        target_date = self.date_slider.value
        
        if route_id is None:
            return
        
        # Create and display map
        map_obj = self.create_map(route_id, target_date)
        
        # Display route lifetime info
        start_date, end_date = self.get_route_lifetime(route_id)
        route_info = self.routes[self.routes['route_id'] == route_id]
        
        if not route_info.empty:
            route_info = route_info.iloc[0]
            short_name = route_info.get('route_short_name', route_info.get('short_name', str(route_id)))
            long_name = route_info.get('route_long_name', route_info.get('long_name', ''))
            route_name = f"{short_name}" + (f" - {long_name}" if long_name else "")
        else:
            route_name = f"Route {route_id}"
        
        if start_date and end_date:
            pass  # Keep lifetime info silent
        
        # Show main shape info
        main_shape_id = self.get_main_shape_for_date(route_id, target_date)
        
        return map_obj
    
    def display(self):
        """Display the interactive visualization with route details"""
        # Route details output
        details_output = widgets.Output()
        
        display(widgets.VBox([
            self.route_selector,
            self.date_slider
        ]))
        
        # Create output widget for the map
        map_output = widgets.Output()
        display(map_output)
        
        # Display route details after the map
        display(details_output)
        
        def update_display(*args):
            # Update map first
            with map_output:
                clear_output(wait=True)
                map_obj = self.update_visualization()
                if map_obj:
                    display(map_obj)
            
            # Update route details after map
            with details_output:
                clear_output(wait=True)
                route_id = self.route_selector.value
                target_date = self.date_slider.value
                if route_id:
                    details = self.display_route_details(route_id, target_date)
                    print(details)
        
        # Connect the update function
        self.route_selector.observe(update_display, names='value')
        self.date_slider.observe(update_display, names='value')
        
        # Initial display
        update_display()

In [ ]:
# Usage example with automatic folder detection:

# Initialize the visualizer (automatically finds data folder)
visualizer = BudapestRouteVisualizer()

# Display the interactive visualization
visualizer.display()

Output()

Output()